<a href="https://colab.research.google.com/github/jbirondo/2020-FF-Data/blob/master/Week1TargetShareRB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS

def yearly_stats(season=2020):

    BASE_URL = f"https://www.pro-football-reference.com/years/{season}/fantasy.htm"

    res = requests.get(BASE_URL)

    soup = BS(res.content, 'html.parser')

    table = soup.find('table', {'id': 'fantasy'})

    df = pd.read_html(str(table))[0]

    df.columns = df.columns.droplevel(level=0)

    df['PassingTD'] = df['TD'].iloc[:, 0]
    df['PassingYds'] = df['Yds'].iloc[:, 0]
    df['RushingTD'] = df['Yds'].iloc[:, 1]
    df['RushingYds'] = df['Yds'].iloc[:, 1]
    df['ReceivingTD'] = df['TD'].iloc[:, 2]
    df['ReceivingYds'] = df['Yds'].iloc[:, 2]

    df['PassingAtt'] = df['Att'].iloc[:, 0]
    df['RushingAtt'] = df['Att'].iloc[:, 1]

    df = df.rename({
        'FantPos': 'Pos'
    }, axis=1)

    df = df[['Player', 'Tm', 'Pos', 'Age', 'PassingAtt', 'Cmp', 'PassingYds', 'PassingTD', 'RushingAtt', 'RushingYds', 'RushingTD', 'Tgt', 'Rec', 'ReceivingYds', 'ReceivingTD', 'FL', '2PM']]

    df = df.fillna(0)

    df = df.loc[df['Player'] != 'Player'] 

    for column in df.columns[4:]:
        df[column] = df[column].astype(int)

    return df

def get_top_target_share(df, pos='WR', min_receptions=0, n=100):

    df = df.merge(df.groupby('Tm')['Tgt'].sum(), how='left', on='Tm', suffixes=('_ind', '_team'))

    df['Tgt_share'] = df['Tgt_ind'] / df['Tgt_team']

    df = df.loc[(df['Pos'] == pos) & (df['Rec'] >= min_receptions)]

    df = df.sort_values(by='Tgt_share', ascending=False)

    df['Tgt_share_rank'] = df['Tgt_share'].rank(ascending=False, method='min')

    with pd.option_context('display.max_rows', None):
      df = df[['Player', 'Tm', 'Tgt_ind', 'Rec', 'Tgt_share', 'Tgt_share_rank']]
      display(df.head(n))

df = yearly_stats(2020)

get_top_target_share(df, pos='RB', min_receptions=3, n=100)

,Player,Tm,Tgt_ind,Rec,Tgt_share,Tgt_share_rank
26,DeAndre Hopkins,ARI,16,14,0.432432,1.0
1,Davante Adams,GNB,17,14,0.414634,2.0
16,Jamison Crowder,NYJ,13,7,0.393939,3.0
104,Amari Cooper,DAL,14,10,0.378378,4.0
110,Julian Edelman,NWE,7,5,0.368421,5.0
173,Diontae Johnson,PIT,10,6,0.322581,6.0
44,Will Fuller,HOU,10,8,0.322581,6.0
2,Adam Thielen,MIN,8,6,0.320000,8.0
220,N'Keal Harry,NWE,6,5,0.315789,9.0
125,Mike Williams,LAC,9,4,0.310345,10.0
